---
# Praxisteil 3 - Software Architektur des Heuristischen Solvers
---

---
## 1. Zusammenführen der Klassen aus vorherigen Veranstaltungen 
---

Angestrebte Solverarchitektur:

![Solverarchitektur](SolverArchitektur.PNG)

Speichern von allen gefundenen Lösungen in der Klasse `SolutionPool`:

In [ ]:
class SolutionPool:
    def __init__(self):
        self.Solutions = []

    def AddSolution(self, newSolution):
        self.Solutions.append(newSolution)

    def GetLowestMakespanSolution(self):
        self.Solutions.sort(key = lambda solution: solution.Makespan) # sort solutions according to makespan

        return self.Solutions[0]

Alle konstruktiven Verfahren werden in der Klasse `ConstructiveHeuristics` zusammengeführt:

In [ ]:
class ConstructiveHeuristics:
    def __init__(self, evaluationLogic, solutionPool):
        self.RandomSeed = 2021
        self.RandomRetries = 10
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool

    def FirstComeFirstServe(self, jobList):
        tmpPermutation = [*range(len(jobList))]

        tmpSolution = Solution(jobList, tmpPermutation)
        self.EvaluationLogic.DefineStartEnd(tmpSolution)

        return tmpSolution

    # other constructive algorithms

    def Run(self, inputData, solutionMethod):
        print('Generating an initial solution according to ' + solutionMethod + '.')

        solution = None

        if solutionMethod == 'FCFS':
            solution = self.FirstComeFirstServe(inputData.InputJobs)
        elif solutionMethod == 'SPT':
            solution = self.ShortestProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'LPT':
            solution = self.LeastProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'ROS':
            solution = self.ROS(inputData.InputJobs, self.RandomRetries, self.RandomSeed)
        elif solutionMethod == 'NEH':
            solution = self.NEH(inputData.InputJobs)
        else:
            print('Unkown constructive solution method: ' + solutionMethod + '.')

        self.SolutionPool.AddSolution(solution)

Alle bisherigen Elemente werden in einer Solver-Klasse zusammengeführt:

In [ ]:
class Solver:
    def __init__(self, inputData, seed):
        self.InputData = inputData
        self.Seed = seed
        self.RNG = numpy.random.default_rng(seed)

        self.EvaluationLogic = EvaluationLogic(inputData)
        self.SolutionPool = SolutionPool()
        
        self.ConstructiveHeuristic = ConstructiveHeuristics(self.EvaluationLogic, self.SolutionPool)   

    def ConstructionPhase(self, constructiveSolutionMethod):
        self.ConstructiveHeuristic.Run(self.InputData, constructiveSolutionMethod)

        bestInitalSolution = self.SolutionPool.GetLowestMakespanSolution()

        print("Constructive solution found.")
        print(bestInitalSolution)

        return bestInitalSolution

---
## 2. Ausführen des Solvers
---

Damit können wir die konstruktiven Lösungen ROS, FCFS, LPT, SPT Lösung wie vorher erzeugen:

In [ ]:
from Solver import *

data = InputData("InputFlowshopSIST.json")

solver = Solver(data, 2048)

solver.ConstructionPhase('ROS')

---
## 3. Nachbarschaften und Tausche
---
#### Wiederholung aus der Theorie
Nachbarschaften beinhalten eine oder mehrere Lösungen (Nachbarschaftslösungen), die von einer gegebenen Lösung aus mit einem Tausch erreichbar sind.

#### Beispiel 
Gegebene FCFS-Lösung:

In [ ]:
permutationFCFS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
solutionFCFS = Solution(data.InputJobs, permutationFCFS)

solver.EvaluationLogic.DefineStartEnd(solutionFCFS)

print(solutionFCFS)

Nun führen wir einen Swap Tausch (= Move) der Elemente 0 und 1 durch. Das Ergebnis eines Tausches ist nichts anderes als ein neuer Lösungskandidat.

In [ ]:
swapMovePermutation = list(permutationFCFS) # create a copy of the permutation
swapMovePermutation[0] = permutationFCFS[1]
swapMovePermutation[1] = permutationFCFS[0]

swapMoveSolution = Solution(data.InputJobs, swapMovePermutation)

solver.EvaluationLogic.DefineStartEnd(swapMoveSolution)

print(swapMoveSolution)

Dieser eine Tausch in der Swap Nachbarschaft hat bereits zu einer Verbesserung geführt. Um weitere Verbesserungen zu finden ist es nötig die Nachbarschaft weitgehender zu erkunden. Es werden nun alle Tausche in der Swap-Nachbarschaft der Ausgangslösung erstellt und bewertet. Dazu wird die neue Klasse `SwapMove` angelegt.

Formal: $M(\text{swap}, s, i, j)$.

In [ ]:
""" Represents the swap of the element at IndexA with the element at IndexB for a given permutation (= solution). """
class SwapMove:
    def __init__(self, initialPermutation, indexA, indexB):
        self.Permutation = list(initialPermutation) # create a copy of the permutation
        self.IndexA = indexA
        self.IndexB = indexB

        self.Permutation[indexA] = initialPermutation[indexB]
        self.Permutation[indexB] = initialPermutation[indexA]

Nun erstellen wir alle Tausche der Nachbarschaft zur Initiallösung.

Formal: $N(\text{swap}, s) = \bigcup\limits_{i, j, i < j} M(\text{swap}, s, i, j)$

In [ ]:
swapMoves = []
for i in range(len(permutationFCFS)):
    for j in range(len(permutationFCFS)):
        if i < j: # neighborhood is symmetric
            swapMove = SwapMove(permutationFCFS, i, j)
            swapMoves.append(swapMove)

Aus der Theorie wissen wir, dass die Swap-Nachbarschaft symmetrisch ist und eine Komplexität/Größe von $\binom{n}{2}$ hat, wobei $n$ die Anzahl an Jobs ist. Bei 11 Jobs ist also zu erwarten, dass die Nachbarschaft $\binom{11}{2} = 55$ Tausche enthält.

In [ ]:
import math

expectedNumberOfMoves = math.comb(len(data.InputJobs), 2) # binomial coefficient "n choose k"
actualNumberOfMoves = len(swapMoves)

print(f"There should exist {expectedNumberOfMoves} moves, and this is {actualNumberOfMoves == expectedNumberOfMoves}.")

Die neuen Tausche wurden erstellt und damit gibt es 55 neue potenzielle Lösungskandidaten (= Permutationen). Die neuen Lösungskandidaten müssen nun bewertet werden um festzustellen, ob mindestens einer davon zu einer Verbesserung geführt hat. Die Lösungen speichern wir in einer Liste ab, um später darauf zugreifen zu können und den besten gefundenen Lösungskandidaten zu ermitteln.

In [ ]:
swapMoveSolutions = []
for move in swapMoves:
    swapMoveSolution = Solution(data.InputJobs, move.Permutation)

    solver.EvaluationLogic.DefineStartEnd(swapMoveSolution)

    swapMoveSolutions.append(swapMoveSolution)

print("Initial Solution:" + str(permutationFCFS))
for solution in swapMoveSolutions:
    print(solution)

Wir bestimmen die beste gefundene Lösung und überprüfen, ob diese besser ist als die FCFS-Lösung.

In [ ]:
swapMoveSolutions.sort(key = lambda solution: solution.Makespan) # sort candidate solutions according to makespan

bestCurrentSolution = swapMoveSolutions[0]

print(f"New solution: {bestCurrentSolution}")
print(f"FCFS solution: {solutionFCFS}\n")

print(f"The new solution has a makespan that is lower than that of the FCFS solution: {bestCurrentSolution.Makespan < solutionFCFS.Makespan}")

#### Kurze Programmieraufgabe

Es wurde eine bessere Lösung gefunden als die bisherige! Welcher Tausch wurde ausgeführt? Schreiben Sie eine Funktion die das $i$ und $j$ des ausgeführten Tausches $M(\text{swap}, s, i, j)$ ermittelt.

In [ ]:
for index in range(len(permutationFCFS)):
    if permutationFCFS[index] != bestCurrentSolution.Permutation[index]:
        print(index)

Diese Logik gießen wir nun in eine übergeordnete `SwapNeighborhood` Klasse, welche eine Nachbarschaft $N(\text{swap}, s)$ repräsentiert. Wie würden Sie vorgehen? Was sind die einzelnen Bestandteile der Klasse `SwapNeighborhood`? Welche logischen Schritte müssen ausgeführt werden?

In [ ]:
""" Contains all $n choose 2$ swap moves for a given permutation (= solution). """
class SwapNeighborhood:
    def __init__(self, inputData, initialPermutation, evaluationLogic, solutionPool):
        self.InputData = inputData
        self.Permutation = initialPermutation
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool

        self.Moves = []
        self.SwapMoveSolutions = []

    """ Generate all $n choose 2$ moves. """
    def DiscoverMoves(self):
        for i in range(len(self.Permutation)):
            for j in range(len(self.Permutation)):
                if i < j:
                    swapMove = SwapMove(self.Permutation, i, j)
                    self.Moves.append(swapMove)

    """ Evaluate all moves. """
    def EvaluateMovesBestImprovement(self):
        for move in self.Moves:
            swapMoveSolution = Solution(self.InputData.InputJobs, move.Permutation)

            self.EvaluationLogic.DefineStartEnd(swapMoveSolution)

            self.SwapMoveSolutions.append(swapMoveSolution)

    """ Evaluate all moves until the first one is found that improves the best solution found so far. """
    def EvaluateMovesFirstImprovement(self):
        bestObjective = self.SolutionPool.GetLowestMakespanSolution().Makespan

        for move in self.Moves:
            swapMoveSolution = Solution(self.InputData.InputJobs, move.Permutation)

            self.EvaluationLogic.DefineStartEnd(swapMoveSolution)
            self.SwapMoveSolutions.append(swapMoveSolution)

            if swapMoveSolution.Makespan < bestObjective:
                # abort neighborhood evaluation because an improvement has been found
                return

    def EvaluateMoves(self, evaluationStrategy):
        if evaluationStrategy == 'BestImprovement':
            self.EvaluateMovesBestImprovement()
        elif evaluationStrategy == 'FirstImprovement':
            self.EvaluateMovesFirstImprovement()
        else:
            print(f'Evaluation strategy {evaluationStrategy} not implemented.')

    def MakeBestMove(self):
        self.SwapMoveSolutions.sort(key = lambda solution: solution.Makespan) # sort solutions according to makespan

        bestNeighborhoodSolution = self.SwapMoveSolutions[0]

        return bestNeighborhoodSolution

    def Update(self, permutation):
        self.Permutation = permutation

        self.Moves.clear()
        self.SwapMoveSolutions.clear()

    def LocalSearch(self, neighborhoodEvaluationStrategy, solution):
        hasSolutionImproved = True

        while hasSolutionImproved:
            self.Update(solution.Permutation)
            self.DiscoverMoves()
            self.EvaluateMoves(neighborhoodEvaluationStrategy)

            bestNeighborhoodSolution = self.MakeBestMove()

            if bestNeighborhoodSolution.Makespan < solution.Makespan:
                print("New best solution has been found!")
                print(bestNeighborhoodSolution)

                self.SolutionPool.AddSolution(bestNeighborhoodSolution)

                solution.Permutation = bestNeighborhoodSolution.Permutation
                solution.Makespan = bestNeighborhoodSolution.Makespan
            else:
                print(f"Reached local optimum of {self.Type} neighborhood. Stop local search.")
                hasSolutionImproved = False       


---
## 4. Ablauflogik der Verbesserungsverfahren
---

Wir lagern die Logik in das Modul `Neighborhood.py` aus und betten die Nachbarschaften in eine Klasse `ImprovementAlgorithm` ein, welche die Logik des Verbesserungsverfahren steuert.

In [ ]:
""" Base class for several types of improvement algorithms. """ 
class ImprovementAlgorithm:
    def __init__(self, inputData, neighborhoodEvaluationStrategy = 'BestImprovement', neighborhoodTypes = ['Swap']):
        self.InputData = inputData

        self.EvaluationLogic = None
        self.SolutionPool = None
        self.RNG = None

        self.NeighborhoodEvaluationStrategy = neighborhoodEvaluationStrategy
        self.NeighborhoodTypes = neighborhoodTypes
        self.Neighborhoods = {}

    def Initialize(self, evaluationLogic, solutionPool, rng = None):
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool
        self.RNG = rng

    def InitializeNeighborhoods(self, solution):
        self.Neighborhoods['Swap'] = SwapNeighborhood(self.InputData, solution.Permutation, self.EvaluationLogic, self.SolutionPool)
        # add further neighborhoods

Die Klasse `ImprovementAlgorithm` enthält nun alle grundlegenden Komponenten für nachbarschaftsbasierte Verbesserungsverfahren. Diese können in verschiedenen Algorithmen verwendet werden. Hier erstellen wir eine Klasse `IterativeImprovement`, bei der die Lösung iterativ durch eine (sequentielle) lokale Suche verbessert wird.

In [ ]:
""" Iterative improvement algorithm through sequential variable neighborhood descent. """
class IterativeImprovement(ImprovementAlgorithm):
    def __init__(self, inputData, neighborhoodEvaluationStrategy = 'BestImprovement', neighborhoodTypes = ['Swap']):
        super().__init__(inputData, neighborhoodEvaluationStrategy, neighborhoodTypes)

    def Run(self, startSolution):
        self.InitializeNeighborhoods(solution)    

        # According to "Hansen et al. (2017): Variable neighorhood search", this is equivalent to the 
        # sequential variable neighborhood descent with a pipe neighborhood change step.
        for neighborhoodType in self.NeighborhoodTypes:
            neighborhood = self.Neighborhoods[neighborhoodType]

            neighborhood.LocalSearch(self.NeighborhoodEvaluationStrategy, solution)
        
        return solution

Die Klasse `Solver` wird um die Methoden `ImprovementPhase()` und `RunLocalSearch()` erweitert.

In [ ]:
class Solver:
    def __init__(self, inputData, seed):
        # same as before
        pass

    def ConstructionPhase(self, constructiveSolutionMethod):
        # same as before
        pass


    def ImprovementPhase(self, startSolution, algorithm):
        algorithm.Initialize(self.EvaluationLogic, self.SolutionPool, self.RNG)
        bestSolution = algorithm.Run(startSolution)

        print("Best found Solution.")
        print(bestSolution)

    def RunLocalSearch(self, constructiveSolutionMethod, algorithm):
        startSolution = self.ConstructionPhase(constructiveSolutionMethod)

        self.ImprovementPhase(startSolution, algorithm)

In [ ]:
from Solver import *
from ImprovementAlgorithm import *

data = InputData("InputFlowshopSIST.JSON")

solver = Solver(data, 2048)

improvementAlgorithm = IterativeImprovement(data)

solver.RunLocalSearch('FCFS', improvementAlgorithm)

#### Frage
Bisher haben wir zwei Nachbarschaftbewertungsstrategien kennengelernt:
- Best Improvement (extensive Suche): alle Tausche einer Nachbarschaft werden ausgewertet. Der Tausch mit der größten Verbesserung wird ausgeführt.
- First Improvement (selektive Suche): die Tausche einer Nachbarschaft werden solange ausgewertet, bis eine Verbesserung gefunden wird. Sobald eine Verbesserung gefunden wurde, wird die Auswertung abgebrochen und der gefundene Tausch sofort durchgeführt. Bessere Tausche können somit "übersehen" werden.

Was denken Sie, mit welcher Bewertungsstrategie werden bessere Ergebnisse erzielt? Was sind Vor- und Nachteile der ein oder anderen Bewertungsstrategie? Begründen Sie ihre Antwort.

In [ ]:
from Solver import *

data = InputData("InputFlowshopSIST.json")

constructiveSolutionMethod = 'FCFS'

solverBestImprovement = Solver(data, 2048)
bestImprovementAlgorithm = IterativeImprovement(data, 'BestImprovement')
solverBestImprovement.RunLocalSearch(constructiveSolutionMethod, bestImprovementAlgorithm)

solverFirstImprovement = Solver(data, 2048)
bestImprovementAlgorithm = IterativeImprovement(data, 'FirstImprovement')
solverFirstImprovement.RunLocalSearch(constructiveSolutionMethod, bestImprovementAlgorithm)

localOptimumBestImprovement = solverBestImprovement.SolutionPool.GetLowestMakespanSolution()
localOptimumFirstImprovement = solverFirstImprovement.SolutionPool.GetLowestMakespanSolution()

print(f"Best improvement makespan: {localOptimumBestImprovement.Makespan}")
print(f"First improvement makespan: {localOptimumFirstImprovement.Makespan}")

isBestImprovementMakespanLower = localOptimumBestImprovement.Makespan < localOptimumFirstImprovement.Makespan

print(f"The best improvement strategy achieves a lower makespan: {isBestImprovementMakespanLower}")

---
## 5. Wiederholung: gemeinsames Debuggen des Solvers
---

---
## 6. Anwendung: selbständiges Programmieren
---

### Eine neue Nachbarschaft
- Überlegen Sie vorab welche Größe die Insertion Nachbarschaft hat
- Erweitern Sie den Solver um eine Insertion-Nachbarschaft
- Überprüfen Sie, ob Ihre implemtierte Nachbarschaft auch die erwartete Größe aufweist

In [ ]:
from Solver import *

data = InputData("InputFlowshopSIST.json")

improvementAlgorithm = IterativeImprovement(data, 'BestImprovement', ['Swap', 'Insertion'])

solver = Solver(data, 1024)

solver.RunLocalSearch('LPT', improvementAlgorithm)

### Zusatz 
Nun existieren mehrere Nachbarschaften. Überlegen Sie wie man einen sinnvollen Wechsel zwischen den Nachbarschaften steuern kann. Momentan ist in der Intesivierungsphase ein Sequential Variable Neighborhood Descent mit einem Pipe Neighborhood Change Step implementiert. Vollziehen Sie die Richtigkeit dieser Aussage nach.

- Implementieren Sie einen der anderen vorgestellten Neighborhood Change Steps: "Basic"
- Welche Vor- und Nachteile erwarten Sie dadurch?
- Welche Schwierigkeit erwarten Sie bei der Umsetzung eines "Cyclic" Neighborhood Change Steps?